Fase 2 - Análise de Dados de Jogos da Steam

In [ ]:
import sys
import os

# Adiciona o diretório raiz do projeto ao PYTHONPATH
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from src import DataLoader